In [2]:
import argparse
import copy
import os
os.chdir('/root/bevfusion/')

import mmcv
import numpy as np
import torch
from mmcv import Config
from mmcv.parallel import MMDistributedDataParallel
from mmcv.runner import load_checkpoint
from torchpack import distributed as dist
from torchpack.utils.config import configs
# from torchpack.utils.tqdm import tqdm # this gives an import error
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

In [3]:
# These take time to load
from mmdet3d.core import LiDARInstance3DBoxes
from mmdet3d.core.utils import visualize_camera, visualize_lidar, visualize_map
from mmdet3d.datasets import build_dataloader, build_dataset
from mmdet3d.models import build_model


def recursive_eval(obj, globals=None):
    if globals is None:
        globals = copy.deepcopy(obj)

    if isinstance(obj, dict):
        for key in obj:
            obj[key] = recursive_eval(obj[key], globals)
    elif isinstance(obj, list):
        for k, val in enumerate(obj):
            obj[k] = recursive_eval(val, globals)
    elif isinstance(obj, str) and obj.startswith("${") and obj.endswith("}"):
        obj = eval(obj[2:-1], globals)
        obj = recursive_eval(obj, globals)

    return obj

In [ ]:
os.environ['MASTER_HOST'] = 'localhost:50091' # We need to specify some port when running from a jupyter notebook
# When you do it using the torchpack dist-run command, it takes care of that automatically
dist.init()

In [ ]:
args = argparse.Namespace(bbox_classes=None, bbox_score=None, checkpoint='/pretrain/bevfusion-det.pth', config='configs/nuscenes/det/transfusion/secfpn/camera+lidar/swint_v0p075/convfuser.yaml', map_score=0.5, mode='pred', out_dir='./visualization', split='val')
opts = ['--box-score', '0.1']

configs.load(args.config, recursive=True)
configs.update(opts)

In [ ]:
cfg = Config(recursive_eval(configs), filename=args.config)

torch.backends.cudnn.benchmark = cfg.cudnn_benchmark

# build the dataloader
dataset = build_dataset(cfg.data[args.split])
dataflow = build_dataloader(
	dataset,
	samples_per_gpu=1,
	workers_per_gpu=cfg.data.workers_per_gpu,
	dist=True,
	shuffle=False,
)

In [7]:
# Build the model
model = build_model(cfg.model)
load_checkpoint(model, args.checkpoint, map_location="cpu")

model = MMDistributedDataParallel(
    model.cuda(),
    device_ids=[torch.cuda.current_device()],
    broadcast_buffers=False,
)
model.eval()

/opt/conda/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1639180588308/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
2022-11-27 17:40:40,573 - mmdet - INFO - load checkpoint from http path: https://github.com/SwinTransformer/storage/releases/download/v1.0.0/swin_tiny_patch4_window7_224.pth


load checkpoint from local path: /pretrain/bevfusion-det.pth


MMDistributedDataParallel(
  (module): BEVFusion(
    (encoders): ModuleDict(
      (camera): ModuleDict(
        (backbone): SwinTransformer(
          (patch_embed): PatchEmbed(
            (adap_padding): AdaptivePadding()
            (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
            (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
          )
          (drop_after_pos): Dropout(p=0.0, inplace=False)
          (stages): ModuleList(
            (0): SwinBlockSequence(
              (blocks): ModuleList(
                (0): SwinBlock(
                  (norm1): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
                  (attn): ShiftWindowMSA(
                    (w_msa): WindowMSA(
                      (qkv): Linear(in_features=96, out_features=288, bias=True)
                      (attn_drop): Dropout(p=0.0, inplace=False)
                      (proj): Linear(in_features=96, out_features=96, bias=True)
                      (proj_dro

For the detection: outputs[0].keys() -> dict_keys(['boxes_3d', 'scores_3d', 'labels_3d'])

In [8]:
print(cfg.object_classes)

['car', 'truck', 'construction_vehicle', 'bus', 'trailer', 'barrier', 'motorcycle', 'bicycle', 'pedestrian', 'traffic_cone']


In [9]:
counter = 0
datalist = []
for data in dataflow:
	datalist.append(data)
	if counter > 5:
		break
	counter += 1

/opt/conda/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int_coords(poly.exterior.coords) for poly in polygons]
/opt/conda/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1824: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  interiors = [int_coords(pi.coords) for poly in polygons for pi in poly.interiors]
/opt/conda/lib/python3.8/site-packages/nuscenes/map_expansion/map_api.py:1823: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  exteriors = [int

In [47]:
import time
counter = 0
for i, data in enumerate(datalist):
    if i >= 2:
        break

    metas = data["metas"].data[0][0]
    name = "{}-{}".format(metas["timestamp"], metas["token"])

    with torch.inference_mode():
        start = time.time()
        outputs = model(**data) # Made predictions
        print(time.time() - start)

    gt_bboxes = data["gt_bboxes_3d"].data[0][0].tensor.numpy()
    gt_labels = data["gt_labels_3d"].data[0][0].numpy()

    gt_bboxes[..., 2] -= gt_bboxes[..., 5] / 2
    gt_bboxes = LiDARInstance3DBoxes(gt_bboxes, box_dim=9)

    # elif args.mode == "pred" and "boxes_3d" in outputs[0]:
    bboxes = outputs[0]["boxes_3d"].tensor.numpy()
    scores = outputs[0]["scores_3d"].numpy()
    labels = outputs[0]["labels_3d"].numpy()

    # Remove boxes from threshold you don't care about
    BBOX_SCORE = 0.1
    indices = scores >= BBOX_SCORE
    bboxes = bboxes[indices]
    scores = scores[indices]
    labels = labels[indices]
    
    
    bboxes[..., 2] -= bboxes[..., 5] / 2 # I don't know why they do this
    bboxes = LiDARInstance3DBoxes(bboxes, box_dim=9)
    
    # For BEV Models
    if args.mode == "gt" and "gt_masks_bev" in data:
        masks = data["gt_masks_bev"].data[0].numpy()
        masks = masks.astype(np.bool)
    elif args.mode == "pred" and "masks_bev" in outputs[0]:
        masks = outputs[0]["masks_bev"].numpy()
        masks = masks >= args.map_score
    else:
        masks = None
    
    if "img" in data:
        for k, image_path in enumerate(metas["filename"]):
            image = mmcv.imread(image_path)
            visualize_camera( # This does some extra preprocessing for some reason, 
                os.path.join(args.out_dir, f"camera-{k}", f"{labels}-{name}.png"),
                image,
                bboxes=bboxes,
                labels=labels,
                transform=metas["lidar2image"][k],
                classes=cfg.object_classes,
            )
        
        final_image = Image.open(os.path.join(args.out_dir, f"camera-0", f"{labels}-{name}.png"))
        final_image.show()
        print(scores)
        print([cfg.object_classes[label] for label in labels])
        print(bboxes)
        # plt.imshow(final_image)
        # plt.axis('off')
    
    # if "points" in data:
    #     lidar = data["points"].data[0][0].numpy()
    #     visualize_lidar(
    #         os.path.join(args.out_dir, "lidar", f"{name}.png"),
    #         lidar,
    #         bboxes=bboxes,
    #         labels=labels,
    #         xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
    #         ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
    #         classes=cfg.object_classes,
    #     )

    # if masks is not None:
    #     visualize_map(
    #         os.path.join(args.out_dir, "map", f"{name}.png"),
    #         masks,
    #         classes=cfg.map_classes,
    #     )



0.24267315864562988
